In [1]:
#This notebook builds a ensemble from baseline randomforest on pubchem_assays using 
#Morgan fingerprint 
#and cell painting

In [2]:
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from scipy import stats
from rdkit.Chem import rdMolDescriptors as rdmd
from rdkit.Chem.Scaffolds import MurckoScaffold
import pandas as pd
from tqdm import tqdm
import time
import numpy as np
from scipy.spatial.distance import cdist
from sklearn.cluster import MiniBatchKMeans
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef,confusion_matrix, roc_auc_score, roc_curve
import seaborn as sns

In [3]:
list_of_lists_df = pd.read_csv("Predictions_train_heldout_scaled_prob_all_assays.csv")
list_of_lists_df

,assay,StdInChI,fp_proba,fp_pred,fp_threshold,CP_proba,CP_pred,CP_threshold,true,ts,pc,Data,MFP_Correct,CP_Correct,fp_proba_scaled,CP_proba_scaled
0,588458,"InChI=1S/C19H19N3O5S3/c23-16-6-5-13(30(25,26)2...",0.215985,0,0.278423,0.291141,0,0.306456,0.0,0.418605,0.430046,Training,True,True,0.387873,0.475014
1,588458,InChI=1S/C9H8ClN3S/c1-6-11-12-9(14)13(6)8-4-2-...,0.240910,0,0.278423,0.253367,0,0.306456,0.0,0.409091,0.680534,Training,True,True,0.432633,0.413382
2,588458,InChI=1S/C20H20N2O4/c1-2-25-20(24)17-12-18-16(...,0.232871,0,0.278423,0.154150,0,0.306456,0.0,0.447368,0.701574,Training,True,True,0.418196,0.251504
3,588458,InChI=1S/C26H24N2O5S/c1-33-19-11-9-18(10-12-19...,0.142133,0,0.278423,0.359792,1,0.306456,1.0,0.341463,0.453313,Training,False,True,0.255247,0.538452
4,588458,InChI=1S/C19H21N3O3S/c1-13-10-16-11-15(4-9-19(...,0.253998,0,0.278423,0.202499,0,0.306456,0.0,0.388889,0.646760,Training,True,True,0.456137,0.330388
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50467,1117305,InChI=1S/C12H9F3N2O2/c1-7-10(6-16-19-7)11(18)1...,0.340631,0,0.357222,0.357490,1,0.330242,0.0,0.296296,0.479437,HeldOut,True,False,0.476778,0.520342
50468,1117305,InChI=1S/C12H10N2/c1-8-12-10(6-7-13-8)9-4-2-3-...,0.242578,0,0.357222,0.244216,0,0.330242,0.0,0.363636,0.619949,HeldOut,True,True,0.339534,0.369752
50469,1117305,InChI=1S/C8H4Cl2N2O2/c9-3-1-5-6(2-4(3)10)12-8(...,0.403774,1,0.357222,0.352388,1,0.330242,1.0,0.333333,0.541732,HeldOut,True,True,0.536212,0.516533
50470,1117305,InChI=1S/C15H12O5/c16-9-3-1-8(2-4-9)13-7-12(19...,0.478660,1,0.357222,0.243638,0,0.330242,0.0,0.444444,0.506977,HeldOut,False,True,0.594464,0.368877


In [4]:
list_of_lists_df["simple_pred"]=round((list_of_lists_df["fp_proba_scaled"] + list_of_lists_df["CP_proba_scaled"])/2)
list_of_lists_df["Ensemble_Correct"] = list_of_lists_df["simple_pred"]==list_of_lists_df["true"]
list_of_lists_df

,assay,StdInChI,fp_proba,fp_pred,fp_threshold,CP_proba,CP_pred,CP_threshold,true,ts,pc,Data,MFP_Correct,CP_Correct,fp_proba_scaled,CP_proba_scaled,simple_pred,Ensemble_Correct
0,588458,"InChI=1S/C19H19N3O5S3/c23-16-6-5-13(30(25,26)2...",0.215985,0,0.278423,0.291141,0,0.306456,0.0,0.418605,0.430046,Training,True,True,0.387873,0.475014,0.0,True
1,588458,InChI=1S/C9H8ClN3S/c1-6-11-12-9(14)13(6)8-4-2-...,0.240910,0,0.278423,0.253367,0,0.306456,0.0,0.409091,0.680534,Training,True,True,0.432633,0.413382,0.0,True
2,588458,InChI=1S/C20H20N2O4/c1-2-25-20(24)17-12-18-16(...,0.232871,0,0.278423,0.154150,0,0.306456,0.0,0.447368,0.701574,Training,True,True,0.418196,0.251504,0.0,True
3,588458,InChI=1S/C26H24N2O5S/c1-33-19-11-9-18(10-12-19...,0.142133,0,0.278423,0.359792,1,0.306456,1.0,0.341463,0.453313,Training,False,True,0.255247,0.538452,0.0,False
4,588458,InChI=1S/C19H21N3O3S/c1-13-10-16-11-15(4-9-19(...,0.253998,0,0.278423,0.202499,0,0.306456,0.0,0.388889,0.646760,Training,True,True,0.456137,0.330388,0.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50467,1117305,InChI=1S/C12H9F3N2O2/c1-7-10(6-16-19-7)11(18)1...,0.340631,0,0.357222,0.357490,1,0.330242,0.0,0.296296,0.479437,HeldOut,True,False,0.476778,0.520342,0.0,True
50468,1117305,InChI=1S/C12H10N2/c1-8-12-10(6-7-13-8)9-4-2-3-...,0.242578,0,0.357222,0.244216,0,0.330242,0.0,0.363636,0.619949,HeldOut,True,True,0.339534,0.369752,0.0,True
50469,1117305,InChI=1S/C8H4Cl2N2O2/c9-3-1-5-6(2-4(3)10)12-8(...,0.403774,1,0.357222,0.352388,1,0.330242,1.0,0.333333,0.541732,HeldOut,True,True,0.536212,0.516533,1.0,True
50470,1117305,InChI=1S/C15H12O5/c16-9-3-1-8(2-4-9)13-7-12(19...,0.478660,1,0.357222,0.243638,0,0.330242,0.0,0.444444,0.506977,HeldOut,False,True,0.594464,0.368877,0.0,True


In [8]:
list_of_lists_df.to_csv("Predictions_train_heldout_scaled_prob_all_assays_ensemble.csv", index=False)